In [7]:
import duckdb as db
import re

In [8]:
 # Connect to sql database
con = db.connect(database='../data/database/german-parliament.duckdb', read_only=False)

In [4]:
con.execute("select * from group_mention limit 5").fetchdf()

,id,paragraph_no,speech_id,paragraph,group_text,label
0,1,0,ID197001700,Wir gehen mit Frankreich ambitioniert nach vor...,Frankreich,GPE
1,2,0,ID197001700,Wir gehen mit Frankreich ambitioniert nach vor...,dem Europäischen Rat,EOPOL
2,3,0,ID197001700,Wir gehen mit Frankreich ambitioniert nach vor...,der Spediteure,PFUNK
3,4,0,ID197001700,Wir gehen mit Frankreich ambitioniert nach vor...,Europa,GPE
4,5,0,724931,Die gesamte Bundesregierung teilt die Aussage ...,Die gesamte Bundesregierung,EOPOL


In [5]:
if paragraph and group:
    essential_label_content = re.sub(r'\s', '', group)
    if essential_label_content:
        # Construct the regex pattern
        # For case-insensitive, add re.IGNORECASE flag to re.finditer
        # e.g., re.finditer(pattern, paragraph, re.IGNORECASE)
        pattern = r'\s*'.join([re.escape(char) for char in essential_label_content])
        
        # Iterate over all non-overlapping matches
        for match_obj in re.finditer(pattern, paragraph):
            start_index = match_obj.start()
            end_index = match_obj.end() # Correct end_index from the regex match
            print(f"start_index: {start_index}, end_index: {end_index}")
            # Optionally, to see the matched text:
            # print(f"  Matched text: '{paragraph[start_index:end_index]}'")

NameError: name 'paragraph' is not defined

In [16]:
def create_highlighted_paragraph(paragraph: str, target: str, ignore_case: bool = False):
    """
    @todo docstring
    """

    # Normalize the target by removing all internal whitespace
    essential_target_chars = re.sub(r'\s', '', target)
    
    # Define the base style for the entire paragraph content
    paragraph_base_style = "font-family: 'Figtree Sans Serif', sans-serif; font-size: 16px;"

    # Construct a regex pattern from essential characters, allowing flexible whitespace
    pattern_str = r'\s*'.join([re.escape(char) for char in essential_target_chars])
    
    # The search pattern for re.sub needs to be wrapped in parentheses
    # to create a capturing group for \1 (the matched text from paragraph)
    search_pattern = f"({pattern_str})"

    sub_flags = re.IGNORECASE if ignore_case else 0
    
    # Inline styles specific to the highlight itself
    # (background, font-weight, padding). Font family and size will be inherited.
    highlight_specific_style = "background-color: #FFFF81; font-weight: normal; padding: 0.1em 0;"
    
    replacement_html = fr'<span style="{highlight_specific_style}">\1</span>'
    
    # First, apply highlights to the target words
    highlighted_content = re.sub(
        search_pattern,
        replacement_html,
        paragraph,
        flags=sub_flags
    )
    
    # base style for the entire paragraph content
    paragraph_base_style = "font-family: 'Figtree Sans Serif', sans-serif; font-size: 16px;"
    
    final_styled_paragraph = f'<div style="{paragraph_base_style}">{highlighted_content}</div>'
    
    return final_styled_paragraph

def build_to_be_annotated_data(target_file:str):
    data = con.execute("select * from group_mention where label NOT IN ('EPPOL','EOPOL','GPE','EOWIRT','EOSCI','EOFINANZ','EONGO') order by random() LIMIT 100").fetchdf()
    if not data.empty:
        data['formatted_paragraph'] = data.apply(
        lambda row: create_highlighted_paragraph(row['paragraph'], row['group_text']),
        axis=1  # Apply function to each row
        )
    data..to_csv(target_file, index=False)
    print("Exported to be labeled data to: {target_file}")

def main():
    build_to_be_annotated_data('test_data_to_be_annotated.csv')

if __name__ == "__main__":
    main()


In [17]:
con.close()

In [125]:
con.execute("select * from group_mention where label NOT IN ('EPPOL','EOPOL','GPE','EOWIRT','EOSCI','EOFINANZ','EONGO') order by random() LIMIT 5").fetchdf()


,id,paragraph_no,speech_id,paragraph,group_text,label
0,997,1,848493,Ich greife zur Verdeutlichung einmal eine Fors...,Fast 31 Prozent der Direktoren der Max-Planck-...,PFUNK
1,200,116,582630,Sie haben nicht einmal etwas zum Spitzensteuer...,die verschiedenen Bevölkerungsschichten,PGEN
2,206,122,582630,"Es gehört viel dazu, das Fragment, das die SPD...","Das "" Wall Street Journal",EOMEDIA
3,786,19,633111,"Wir wollen sicherstellen, daß Menschen mit 65 ...",den Menschen,PGEN
4,498,0,632868,Es sind verschiedene Maßnahmen in Vorbereitung...,die Menschen,PGEN
